<a href="https://colab.research.google.com/github/dakilaledesma/NCBG/blob/main/train/CNL_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%time
! unzip -q /content/drive/MyDrive/UNC/NCBG/sept1_classif_training.zip -d /content
! cp /content/drive/MyDrive/UNC/NCBG/qry_fsus_trainingdata_ai.xlsx /content/qry_fsus_trainingdata_ai.xlsx

CPU times: user 43.2 ms, sys: 13.7 ms, total: 56.9 ms
Wall time: 4.15 s


In [14]:
! mv /content/content/images /content/images
! rm -rf /content/content

In [15]:
from glob import glob
import os
import shutil
import pandas as pd

df = pd.read_excel("qry_fsus_trainingdata_ai.xlsx")

images = glob("images/*.*")
anno_to_fn = {}
for im_fn in images:
  bn = os.path.basename(im_fn).split("__")[1].replace(".jpg", '')
  anno_to_fn[f"{bn}.jpg"] = im_fn

# print(list(anno_to_fn.items())[:5])

for idx, row in df.iterrows():
  anno_bn = row["file"]
  try:
    im_fn = anno_to_fn[anno_bn]
  except KeyError:
    pass
  im_qual = row["imagequality"]

  if not os.path.isdir(f"qual/{im_qual}"):
    os.makedirs(f"qual/{im_qual}")

  try:
    shutil.move(im_fn, f"qual/{im_qual}/{idx}_{anno_bn}")
  except FileNotFoundError:
    pass

In [16]:
! rm -rf /content/qual/nan

In [18]:
! mv /content/qual/1.0 /content/qual/0
! mv /content/qual/5.0 /content/qual/1
! mv /content/qual/10.0 /content/qual/2

In [19]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 11006, done.
remote: Counting objects: 100% (533/533), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 11006 (delta 333), reused 378 (delta 250), pack-reused 10473
Receiving objects: 100% (11006/11006), 20.54 MiB | 22.52 MiB/s, done.
Resolving deltas: 100% (8048/8048), done.


In [20]:
import fileinput
import sys

def replacement(file, previousw, nextw):
   for line in fileinput.input(file, inplace=1):
       line = line.replace(previousw, nextw)
       sys.stdout.write(line)

file = "/content/pytorch-image-models/timm/utils/checkpoint_saver.py"
replacement(file, "if os.path.exists(last_save_path):", "# if os.path.exists(last_save_path):")
replacement(file, "os.unlink(last_save_path)  # required for Windows support.", "# os.unlink(last_save_path)  # required for Windows support.")
replacement(file, "os.link(last_save_path, save_path)", "# os.link(last_save_path, save_path)")
replacement(file, "os.unlink(best_save_path)", "# os.unlink(best_save_path)")
replacement(file, "os.link(last_save_path, best_save_path)", "# os.link(last_save_path, best_save_path)")
replacement(file, "if os.path.exists(best_save_path):", "# if os.path.exists(best_save_path):")


In [21]:
from glob import glob

print(len(list(glob("qual/*"))))

3


In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py qual --model convnext_large_384_in22ft1k --opt adabelief --lr 0.00005 --epochs 80 --decay-epochs 3 --cooldown-epochs 0 --weight-decay 1e-4 --sched cosine -b 4 --input-size 3 600 600 --num-classes=3 --vflip 0.5 --hflip 0.5 --amp --pretrained --output drive/MyDrive/UNC/NCBG/output/ 

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Training with a single process on 1 GPUs.
Loading pretrained weights from url (https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_1k_384.pth)
Model convnext_large_384_in22ft1k created, param count:196234947
Data processing configuration for current model + dataset:
	input_size: (3, 600, 600)
	interpolation: bicubic
	mean: (0.485, 0.456, 0.406)
	std: (0.229, 0.224, 0.225)
	crop_pct: 1.0
Using native Torch AMP. Training in mixed precision.
Scheduled epochs: 80
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dat